In [1]:
import os
import shutil

# Определение исходной директории и целевой директории
source_directory = "/kaggle/input/yolo-small-data"  # Замените 'your-dataset-name' на имя вашего датасета
destination_directory = "/kaggle/working/yolo-small-data"  # Имя целевой директории в /kaggle/working/

# Создание целевой директории, если она не существует
os.makedirs(destination_directory, exist_ok=True)

# Функция для копирования файлов и директорий
def copy_files(source_dir, destination_dir):
    for item in os.listdir(source_dir):
        source_path = os.path.join(source_dir, item)
        destination_path = os.path.join(destination_dir, item)

        if os.path.isdir(source_path):
            shutil.copytree(source_path, destination_path)  # Копирование директорий
        else:
            shutil.copy2(source_path, destination_path)  # Копирование файлов

# Вызов функции копирования
copy_files(source_directory, destination_directory)

In [2]:
import yaml

# Путь к вашему файлу data.yaml
yaml_file_path = "/kaggle/input/yolo-small-data/data.yaml"  # Замените 'your-dataset-name' на имя вашего датасета

# Чтение содержимого файла data.yaml
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Изменение нужных строк
data['train'] = '/kaggle/working/yolo-small-data/train/images'  # Пример изменения пути к тренировочным изображениям
data['val'] = '/kaggle/working/yolo-small-data/valid/images'    # Пример изменения пути к валидационным изображениям

# Путь к новому файлу data.yaml, который будет сохранен в директории working
new_yaml_file_path = "/kaggle/working/yolo-small-data/data.yaml"

# Сохранение изменений в новом файле
with open(new_yaml_file_path, 'w') as file:
    yaml.safe_dump(data, file)

print(f"Измененный файл сохранен как {new_yaml_file_path}")


Измененный файл сохранен как /kaggle/working/yolo-small-data/data.yaml


In [3]:
import os
import shutil
def copy_files(source_dir, destination_dir_1, destination_dir_2):
  """Copies all files from a source directory to a destination directory.

  Args:
    source_dir: The path to the source directory.
    destination_dir: The path to the destination directory.
  """
  counter = 0

  for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    if counter < 3500:
        destination_path = os.path.join(destination_dir_1, filename)
    else:
        destination_path = os.path.join(destination_dir_2, filename)

    if os.path.isfile(source_path):
      shutil.copy2(source_path, destination_path)
    
    counter+=1

# Example usage:
source_directory = "/kaggle/input/nosmoking/other"
destination_directory_1 = "/kaggle/working/yolo-small-data/train/images"
destination_directory_2 = "/kaggle/working/yolo-small-data/valid/images"
copy_files(source_directory, destination_directory_1, destination_directory_2)

In [4]:
ANNOTATIONS_DIRECTORY_PATH = "/kaggle/working/yolo-small-data/train/labels"
IMAGES_DIRECTORY_PATH = "/kaggle/working/yolo-small-data/train/images"
DATA_YAML_PATH = "/kaggle/working/yolo-small-data/data.yaml"

In [5]:
!pip install autodistill autodistill-florence-2 roboflow supervision==0.9.0 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 17.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187315346 sha256=6ebfbdcbdd164f80278a954d29a1bc9d620130264215f0fb93374a6ab4e0a283
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [6]:
import supervision as sv

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset)

1617

In [10]:
import os

# Укажите путь к папке, в которой хотите посчитать файлы
folder_path = IMAGES_DIRECTORY_PATH  # Замените на нужный вам путь

# Подсчет количества файлов в папке
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Количество файлов в папке '{folder_path}': {file_count}")


Количество файлов в папке '/kaggle/working/yolo-small-data/train/images': 5117


In [14]:
SAMPLE_SIZE = 64
SAMPLE_GRID_SIZE = (9, 4)
SAMPLE_PLOT_SIZE = (16, 16)
image_names = list(dataset.images.keys())[144:144+63]

mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()

images = []
for image_name in image_names:
    image = dataset.images[image_name]
    annotations = dataset.annotations[image_name]
    labels = [
        dataset.classes[class_id]
        for class_id
        in annotations.class_id]
    annotates_image = mask_annotator.annotate(
        scene=image.copy(),
        detections=annotations)
    annotates_image = box_annotator.annotate(
        scene=annotates_image,
        detections=annotations,
        labels=labels)
    images.append(annotates_image)

sv.plot_images_grid(
    images=images,
    titles="1",
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

ValueError: The number of images exceeds the grid size. Please increase the grid size or reduce the number of images.

In [12]:
!pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 21.3 MB/s eta 0:00:0000:01


In [13]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')  # здесь можно попробовать другой размер вместо n (s, m, l, x)

# Use the model
results = model.train(data='/kaggle/working/yolo-small-data/data.yaml', epochs=100)  # train the model

100%|██████████| 21.5M/21.5M [00:00<00:00, 105MB/s] 


Ultralytics YOLOv8.2.84 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/yolo-small-data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 20.1MB/s]
2024-08-31 11:56:32,956	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-31 11:56:33,913	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

                   all       1131        507      0.738        0.6      0.671      0.443



100 epochs completed in 3.057 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.84 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  83%|████████▎ | 30/36 [00:08<00:02,  2.04it/s]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:11<00:00,  3.12it/s]


                   all       1131        507      0.738      0.601      0.671      0.443
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▁▁▂▃▃▃▄▄▄▅▅▅▆▅▆▆▆▇▆▇▇▇▇▇▇▇▇███████████
metrics/mAP50-95(B),▁▁▁▁▁▂▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████
metrics/precision(B),▂▁▂▂▂▄▃▄▄▄▄▅▅▆▅▅▅▆▆▇▇▇▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇██
metrics/recall(B),▂▇▁▁▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█▇█▇█▇████▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▄█▇▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
